In [ ]:
# conda activate sdvlm
# pip install jupyterlab
# jupyter lab

In [1]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.mm_utils import tokenizer_image_token, process_images
from llava.constants import IMAGE_TOKEN_INDEX

from PIL import Image
import torch

model_path = "cpystan/SD-VLM-7B"

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_name=get_model_name_from_path(model_path),
    model_base=None,
    load_8bit=True,
    load_4bit=False,
    mm_vision_tower="openai/clip-vit-large-patch14-336"
)
model.gt_depth = False

/home/than/miniconda3/envs/sdvlm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
xFormers not available
xFormers not available
/home/than/miniconda3/envs/sdvlm/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|█| 4/4 [00:03<00:00,  1.11it/s
Some weights of the model checkpoint at cpystan/SD-VLM-7B were not used when initializing LlavaLlamaForCausalLM: ['depth.pretrained.blocks.0.ls1.weight', 'depth.pretrained.blocks.0.ls2.weight', 'depth.pretrained.blocks.1.ls1.weight', 'depth.pretrained.blocks.1.ls2.weight', 'depth.pretrained.blocks.10.ls1.weight', 'dep

In [2]:
#prompt = "What is the width of this table?"
prompt = "What are the dimenions of this table in meters?"
image_file = "example_data/table.jpg"

input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
image = Image.open(image_file).convert('RGB')
image_tensor = process_images([image], image_processor, model.config)[0]

In [3]:
temperature = 0.2
with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=image_tensor.unsqueeze(0).half().cuda(),
        image_sizes=[image.size],
        do_sample=True if temperature > 0 else False,
        temperature=temperature,
        top_p=None,
        num_beams=1,
        ori_imgs = [image],
        max_new_tokens=1024,
        use_cache=True,)

response = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print(response)

The dimensions of the table are 1.41 m x 0.72 m x 0.77 m.


In [4]:
from datasets import load_dataset

In [5]:
data_path = "cpystan/MSMU"
data = load_dataset(data_path, streaming=False)['test']
#data = load_dataset(data_path, streaming=False)['train']  # Might need to stream -- its big!

In [ ]:
import json
for i in range(len(data)):
    print(i)
    sample = data[i]
    # HACK
    c = sample["conversations"]
    c = c.replace("'from': 'human'", "\"from\": \"human\"")
    c = c.replace("'from': 'gpt'", "\"from\": \"gpt\"")
    c = c.replace("'value': '", "\"value\": \"")
    c = c.replace("'value'", "\"value\"")
    c = c.replace("'}", "\"}")
    c = c.replace("}", "},")
    c = c.replace("},]", "}]")
    conversations = json.loads(c)
    prompt = conversations[0]['value']
    print("PROMPT", prompt)
    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
    
    image = sample["image"]
    image_tensor = process_images([image], image_processor, model.config)[0]
    
    temperature = 0.2
    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=image_tensor.unsqueeze(0).half().cuda(),
            image_sizes=[image.size],
            do_sample=True if temperature > 0 else False,
            temperature=temperature,
            top_p=None,
            num_beams=1,
            ori_imgs = [image],
            max_new_tokens=1024,
            use_cache=True,)
    
    response = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
    print("RESPONSE", response)
    ground_truth = conversations[1]['value']
    print("GNDTRUTH", ground_truth)

0
RESPONSE The width  of the wall shelves is 0.4 meters.
GNDTRUTH the wall shelves measures 0.37 meters in width.
1
RESPONSE The width  of the cluttered shelf is 0.45 meters.
GNDTRUTH the cluttered shelf measures 54.91 cm in width.
2
RESPONSE the cluttered shelf is more closer.
GNDTRUTH the office chair is more closer.
3
RESPONSE The distance is 0.86 m.
GNDTRUTH the trash can and the wooden bookshelf are 0.7 m apart from each other.
4
RESPONSE The height of the black chair is 0.81 m and the wooden bookshelf is 0.8 meters, so the black chair is taller.
GNDTRUTH The height of the black chair is 78.14 cm and the wooden bookshelf is 118.68 cm, so the wooden bookshelf is taller.
5
RESPONSE The height of the wooden bookshelf is 1.03 meters.
GNDTRUTH The height of the wooden bookshelf is 1.19 meters.
6
RESPONSE No window in the scene.
GNDTRUTH No window in the scene.
7
RESPONSE No, the dark clothing rack is to the left of the gray curtain.
GNDTRUTH Indeed, the dark clothing rack is positioned